# 루브릭
1. 데이터의 전처리 및 구성과정이 체계적으로 진행되었는가?
    - 특수문자 제거, 토크나이저 생성, 패딩 처리의 작업들이 빠짐없이 진행되었는가?
2. 가사 텍스트 생성 모델이 정상적으로 작동하는가?
    - 텍스트 제너레이션 결과로 생성된 문장이 해석 가능한 문장인가?
3. 텍스트 생성모델이 안정적으로 학습되었는가?
    - 텍스트 생성모델의 validation loss가 2.2이하로 낮아졌는가?

### 필요한 라이브러리 불러오기

In [1]:
import glob
import os
import re
import tensorflow as tf

### 데이터 불러오기

In [2]:
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*' 

txt_list = glob.glob(txt_file_path) 

raw_corpus = [] 

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['', '', '[Spoken Intro:]']


1. 모두 소문자로 변경, 공백 지우기
2. 특수문자 양쪽에 공백 넣기
3. 여러 공백 -> 하나의 공백
4. a-zA-Z.!,?가 아닌 모든 문자를 하나의 공백으로 바꿈
5. 다시 양쪽 공백을 지움
6. 문장 시작 <start>, 끝 <end> 추가
    + 토큰화 했을 때, 토큰의 개수가 15개를 넘어가면 학습데이터에서 제외하기

In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()   #1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)   #2
    sentence = re.sub(r'[" "]+', " ", sentence)  #3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  #4
    sentence = sentence.strip()   #5
    sentence_check = sentence.split(' ')      
    
    if len(sentence_check) > 13:         #6
        return 0
    else:
        sentence = '<start> ' + sentence + ' <end>'
        return sentence 
    

In [4]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0:
        continue
    if sentence[-1] == ":":
        continue
        
    preprocessed_sentence = preprocess_sentence(sentence)
    if preprocessed_sentence == 0:
        pass
    else:
        corpus.append(preprocessed_sentence)
    
corpus[:10]

['<start> spoken intro <end>',
 '<start> you ever want something <end>',
 '<start> that you know you shouldn t have <end>',
 '<start> the more you know you shouldn t have it , <end>',
 '<start> the more you want it <end>',
 '<start> and then one day you get it , <end>',
 '<start> it s so good too <end>',
 '<start> but it s just like my girl <end>',
 '<start> when she s around me <end>',
 '<start> i just feel so good , so good <end>']

In [5]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=12000, filters=' ', oov_token="<unk")
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    
    print(tensor, tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2 2711 2353 ...    0    0    0]
 [   2    7  161 ...    0    0    0]
 [   2   15    7 ...    0    0    0]
 ...
 [   2   41    6 ...    0    0    0]
 [   2   31    7 ...    0    0    0]
 [   2  302    1 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fa8cf5214f0>


In [6]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])
    
    if idx >= 10:
        break

1 : <unk
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to


In [7]:
src_input = tensor[:, :-1]
tg_input = tensor[:, 1:]

print(src_input)
print(tg_input)

[[   2 2711 2353 ...    0    0    0]
 [   2    7  161 ...    0    0    0]
 [   2   15    7 ...    0    0    0]
 ...
 [   2   41    6 ...    3    0    0]
 [   2   31    7 ...    3    0    0]
 [   2  302    1 ...    0    0    0]]
[[2711 2353    3 ...    0    0    0]
 [   7  161   64 ...    0    0    0]
 [  15    7   34 ...    0    0    0]
 ...
 [  41    6  153 ...    0    0    0]
 [  31    7  189 ...    0    0    0]
 [ 302    1 4238 ...    0    0    0]]


In [8]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input)//BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1

dataset = tf.data.Dataset.from_tensor_slices((src_input, tg_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder = True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [9]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tg_input, test_size=0.2, shuffle=True, random_state=42)

print('Source Train :', enc_train.shape)
print('Target Train :', dec_train.shape)

Source Train : (124810, 14)
Target Train : (124810, 14)


In [10]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        #self.rnn_3 = tf.keras.layers.LSTM(hidden_size, return_sequences=True) 
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        #out = self.rnn_3(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 512
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size, hidden_size)

In [11]:
history = [] #loss 값 확인을 위해 history에 저장하자.
epochs = 10

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)

In [12]:
for src_sample, tg_sample in dataset.take(1):break
    
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[-3.08286166e-04, -1.05314815e-04,  2.11993683e-04, ...,
         -1.53507979e-04,  3.36276629e-04,  2.81169545e-04],
        [-4.85586643e-04, -1.20582925e-04,  3.73994291e-04, ...,
         -5.36043372e-04,  5.05706703e-04,  2.86806462e-04],
        [-6.82273065e-04, -1.99608723e-04,  3.99146113e-04, ...,
         -6.30200957e-04,  4.23604157e-04,  3.72720766e-04],
        ...,
        [ 1.63776428e-03,  4.06999665e-04,  1.57505085e-04, ...,
          1.94502776e-04,  3.71674505e-05,  2.33442741e-04],
        [ 2.01546983e-03,  3.29686591e-04,  5.77153056e-04, ...,
          3.45446133e-05, -7.98071851e-05,  1.26645216e-04],
        [ 2.33183615e-03,  2.15672306e-04,  1.03582768e-03, ...,
         -1.25630628e-04, -2.21029986e-04,  1.69234481e-05]],

       [[-3.08286166e-04, -1.05314815e-04,  2.11993683e-04, ...,
         -1.53507979e-04,  3.36276629e-04,  2.81169545e-04],
        [-7.26703613e-04, -2.26518343e-04,  4

In [13]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  6144512   
_________________________________________________________________
lstm (LSTM)                  multiple                  20979712  
_________________________________________________________________
lstm_1 (LSTM)                multiple                  33562624  
_________________________________________________________________
dense (Dense)                multiple                  24590049  
Total params: 85,276,897
Trainable params: 85,276,897
Non-trainable params: 0
_________________________________________________________________


In [14]:
history = model.fit(enc_train, dec_train, epochs=epochs, batch_size=256, validation_data=(enc_val, dec_val), verbose=1)

Epoch 1/10
488/488 [==============================] - 249s 506ms/step - loss: 3.3094 - val_loss: 2.9379
Epoch 2/10
488/488 [==============================] - 250s 512ms/step - loss: 2.7862 - val_loss: 2.7073
Epoch 3/10
488/488 [==============================] - 250s 513ms/step - loss: 2.5113 - val_loss: 2.5350
Epoch 4/10
488/488 [==============================] - 250s 513ms/step - loss: 2.2409 - val_loss: 2.4044
Epoch 5/10
488/488 [==============================] - 251s 514ms/step - loss: 1.9757 - val_loss: 2.3003
Epoch 6/10
488/488 [==============================] - 251s 514ms/step - loss: 1.7258 - val_loss: 2.2232
Epoch 7/10
488/488 [==============================] - 252s 516ms/step - loss: 1.5045 - val_loss: 2.1707
Epoch 8/10
488/488 [==============================] - 252s 516ms/step - loss: 1.3192 - val_loss: 2.1444
Epoch 9/10
488/488 [==============================] - 252s 516ms/step - loss: 1.1762 - val_loss: 2.1439
Epoch 10/10
488/488 [==============================] - 251s 515m

### 모델 평가하기
- 작문 모델을 평가하는 과정에 있어서, 가장 확실한 방법은 작문을 시킨 후 사람이 직접 평가하는 것이므로 텍스트를 생성하여 확인해보자.

In [15]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]
    
    while True:
        predict = model(test_tensor)
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break
            
    generated = ""
    
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "
        
    return generated

In [17]:
Text = generate_text(model, tokenizer, init_sentence="<start> I learn")
print(Text)

<start> i learn to make the worst seem better <end> 


해석을 한다면, "최악의 상황을 더 좋게 만드는 법을 배운다"라고 해석을 할 수 있으다.
따라서 문장을 잘 만들어 낸다고 볼 수 있다.

우선 학습하는 것에 있어 CV에 비해 비교적 오랜 시간의 학습을 한 것 같고 루브릭의 3번째 조건인 validation loss를 2.2 이하를 만들기 위해 여러가지 작업을 진행해 보았다.
- RNN layer를 추가해보았다. 하지만 RNN layer를 추가하니 오히려 2개의 RNN을 가지고 있을 때 보다 성능이 더 낮아지게 되었다.
- embedding size의 값을 여러 수정하였다.데이터 수가 적다면 embedding size가 컸을 때, 좋지 않을 결과 값을 내지만 지금처럼 어느 정도의 데이터가 있다면 충분히 embedding size의 값을 올려가며 실험을 진행하였다.
